In [22]:
import pandas as pd
import os
import matplotlib as plt

In [23]:
files = os.listdir('./')
csv_files = [f for f in files if ".csv" in f]
csv_files

['signals.2018-08-17-222339.0.csv', 'signals.2018-08-18-063826.0.csv']

In [24]:
frame = pd.DataFrame()
list_ = []
for file_ in csv_files:
    df = pd.read_csv(file_,index_col=None, header=None)
    list_.append(df)
frame = pd.concat(list_)
frame.columns = ["ticker", "price", "time", 
                "meta_signal", "previous_meta",
                "4h","1d","1w","1m",
                "4hp","1dp","1wp","1mp"]
frame.head(2)

,ticker,price,time,meta_signal,previous_meta,4h,1d,1w,1m,4hp,1dp,1wp,1mp
0,NASDAQ-AAPL,217.58,1534574531372,2,0,1,2,2,1,0,0,0,0
1,NYSE-ABBV,98.81,1534571111519,1,0,1,2,1,1,0,0,0,0


In [25]:
nvda = frame[frame.ticker.astype(str).str.match("NASDAQ-NVDA")]
nvda

,ticker,price,time,meta_signal,previous_meta,4h,1d,1w,1m,4hp,1dp,1wp,1mp
101,NASDAQ-NVDA,244.82,1534573412624,0,0,-1,-1,1,1,0,0,0,0
